Notebook for Adam experiments

In [ ]:
import datautils
import pandas as pd
import modelutils as mu
import nesterov
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
DATASET = "Dataset/ML-CUP23-TR.csv"
RESULTS = "Results/"
PLOT = "Plots/"

In [ ]:
df_cup = pd.read_csv(DATASET, skiprows=6)
df_cup.rename(columns={"# Training set: ID": "ID"}, inplace=True)

In [ ]:
df_cup.info()

In [ ]:
# create test and training sets
DESIGN_SIZE = 0.8
TRAIN_SIZE = 0.8

df_design, df_test = datautils.hold_out(df_cup, DESIGN_SIZE)

assert (
    len(list(set(df_design["ID"]) & set(df_test["ID"]))) == 0
)  # have intersection of cardinality zero

# TODO: Decidere se fare kfold
df_train, df_val = datautils.hold_out(df_design, TRAIN_SIZE)

df_train.info()

X_train, y_train = datautils.obtain_features_targets(df_train)
print(X_train.shape, y_train.shape)

X_val, y_val = datautils.obtain_features_targets(df_val)
print(X_val.shape, y_val.shape)

X_test, y_test = datautils.obtain_features_targets(df_test)
print(X_test.shape, y_test.shape)

input_size = X_train.shape[1]
output_size = y_train.shape[1]

In [ ]:
# dictionary for results, with keys the strings and values empty lists
results_dict = {
    "Hidden size": [],
    "Mu": [],
    "Beta": [],
    "Epsilon": [],
    "Alpha": [],
    "Seed": [],
    "Initialization type": [],
    "Train": [],
    "Validation": [],
    "Time": [],
    "Epochs": [],
}

In [ ]:
# hyperparameters
HIDDEN_SIZE = 1000
LEARNING_RATE = "auto"
BETA = 0
ALPHA = 1e-8
EPSILON = 1e-10

In [ ]:
# iterate over multiple seeds

for initialization in ["fan-in", "std"]:
    for seed in range(10):
        # initialize the model
        model = mu.ELM(
            input_size,
            HIDDEN_SIZE,
            output_size,
            seed=seed,
            init=initialization,
        )

        # measure the time
        start_time = time.process_time()

        # train model
        final_model, loss_train_history, loss_val_history, epochs = (
            nesterov.nag(
                model,
                X_train,
                y_train,
                X_val,
                y_val,
                lr=LEARNING_RATE,
                alpha=ALPHA,
                beta=BETA,
                max_epochs=1000000,
                eps=EPSILON,
                # fast_mode=True,
                # verbose=True,
            )
        )

        end_time = time.process_time()

        # save the results

        results_dict["Hidden size"].append(HIDDEN_SIZE)
        results_dict["Mu"].append(LEARNING_RATE)
        results_dict["Beta"].append(BETA)
        results_dict["Epsilon"].append(EPSILON)
        results_dict["Alpha"].append(ALPHA)
        results_dict["Seed"].append(seed)
        results_dict["Initialization type"].append(initialization)
        results_dict["Train"].append(loss_train_history[-1])
        results_dict["Validation"].append(loss_val_history[-1])
        results_dict["Time"].append(end_time - start_time)
        results_dict["Epochs"].append(epochs)

        # plot results
        plt.plot(
            np.arange(1, len(loss_train_history) + 1),
            loss_train_history,
            label="Train",
        )
        plt.plot(
            np.arange(1, len(loss_train_history) + 1),
            loss_val_history,
            label="Validation",
        )
        plt.legend()
        # set title and labels
        plt.title(f"Loss {initialization} {seed}")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        # set axis scale
        plt.xscale("log")
        plt.yscale("log")
        # fix y axis to have comparable plots
        plt.ylim(0.001, 1000)
        plt.show()
        # save the plot
        plt.savefig(PLOT + f"loss_{initialization}_{seed}.png")


results = pd.DataFrame(results_dict)
results.to_csv(
    RESULTS + f"results_initialization_{HIDDEN_SIZE}.csv", index=False
)

In [ ]:
#show average training, validation and time, show also the standard deviation
results.groupby("Initialization type")[["Train", "Validation", "Time"]].agg(
    ["mean", "std"]
)